# Linux

## Administrar Usuarios en Linux

* crear usuarios tstark, cdanvers, dprince en servidor
* crear grupo superhero
* hacer usuario tstark  un superusuario
* agregar grupo uperhero a los usuarios tstark, cdanvers, dprince
* bloquear cuenta de usuario dprince


In [ ]:
# abrir consola ssh Linux 

# entrar con usuario ssh
# usuario: cloud_user
# pass: pass
# usuario: cloud_user
# Public IP address of the Server1

ssh usuario@Public IP
password: pass# contraseña


In [ ]:
# agregar usuarios

useradd tstark
useradd dprince
useradd cdanvers 

In [ ]:
# crear grupo

groupadd superhero

In [ ]:
# cambiar grupo wheel a usuario tstark
usermod -g wheel tstark # id usuario

# agregar grupo superhero como grupo suplementario para todos los usuarios
usermod -aG grupo usuario

usermod -aG superhero tstark
usermod -aG superhero dprince
usermod -aG superhero cdanvers

# cada usuario tiene su propio grupo y pertenece al grupo superhero


In [ ]:
# bloquear usuario dprince
usermod -L dprince

## Crear usuarios sudo

* crear 2 usuarios 
* cambiar archivo /etc/sudoers
* probar accesos
* set administrador web 

In [ ]:
# crear usuario sudo
sudo useradd -m gfreeman

# agregar usuario avance
# asignar usuario avance a  grupo wheel suplementario
sudo useradd -G wheel -m avance

# set contraseña de los usuarios:  pass: LASudo321

sudo passwd gfreeman
pass: LASudo321

sudo passwd avance
pass: LASudo321


In [ ]:
# comprobar archivos /etc/sudoers permiten al grupo wheel acceso a los comandos sudo

sudo visudo

In [ ]:
# cambiar a cuenta avance, - para usar login de shell
sudo su - avance

# salir de la cuenta sudo
exit

In [ ]:
# configurar usuario gfreeman para webmaster
# crear nuevo archivo  sudoers en diccionario /etc/sudoers.d con entrada unica para webmasters

sudo visudo -f /etc/sudoers.d/web_admin
Cmnd_Alias  WEB = /bin/systemctl restart httpd.service, 
/bin/systemctl reload httpd.service

# permitir acceso a sudo a usuario gfreeman     
gfreeman ALL=WEB



In [ ]:
# entrar en cuenta gfreeman
sudo su - gfreeman

# intentar reiniciar 
sudo systemctl restart httpd.service

## Uso de Terminal SSH, redireccionamiento y Permisos de Linux

* permitir ssh sin pass de dev@server1 a dev@server2
* copiar todos los archivos .tar de  server1 a server2
* set umask para que los nuevos archivos sean read/write por su propietario solamente
* run el script 'deploy'


In [ ]:
# entrar a ssh con usuario dev y Public IP address
ssh usuario@Public IP address
pass: password servidor1


In [ ]:
# generar key
ssh-keygen

# copiar a  private IP de servidor2
ssh-copy-id  private IP servidor2

# entrar/loggin al servidor2 con IP publica
ssh Public IP servidor2
 

In [ ]:
# copiar todos los archivos .tar de servidor1 a servidor2 de  /home/dev/
scp *.gz server2_PRIVATE_IP:~/

In [ ]:
# ssh conectar a servidor2
ssh <server2_PRIVATE_IP>

# verificar que todos los archivos .tar desde servidor1 fueron copiados 
ll

In [ ]:
# extraer archivos .tar en servidor2
# >>: appending
# >: sobreescribir

tar -xvf deploy_content.tar.gz >> tar-output.log
tar -xvf deploy_script.tar.gz >> tar-output.log

# revisar directorio
ll


In [ ]:
# set unmask en servidor2 para que nuevos archivos sean ONLY READABLE and WRITEABLE por el propietario
umask 0066

# verificar permisos en deploy.sh
# hacer script ejecutable
chmod +x deploy.sh

# run archivo deploy.sh 
./deploy.sh


## Configurar el Boot Target por defecto

In [ ]:
## default target actual es en multi-user

# revisar que target actual es tipo multi-user
systemctl get-default

# cambiar default target para que el computedor arranque(boot) en desktop gráfico
sudo systemctl set-default graphical.target

# revisar que target actual es tipo graphical
systemctl get-default


## Agendar un servicio Systemd con Timer Units


In [ ]:

# entrar a ssh como usuario root
sudo su -

# check directorio actual
pwd

# ver recursos en el directorio
ls

# crear archivo timer unit 
vi web-backup.timer

# codigo de archivo timer unit

[Unit]
Description=Fire off the backup

[Timer]
OnCalendar=*-*-* 23:00:00
Persistent=true
Unit=web-backup.service

[Install]
WantedBy=multi-user.target

# salir de  archivo timer unit
### ESC
### :wq

In [ ]:
# copiar script shell en /usr/local/sbin
cp web-backup.sh /usr/local/sbin/

# copiar servicio y archivos timer en /etc/systemd/
cp web-backup.{service,timer} /etc/systemd/system/

# recargar el Systemd daemon para calcular las dependencias de servicio
systemctl daemon-reload

# habilitar servicios para correr un Boot
systemctl enable web-backup.service
systemctl enable web-backup.timer

# set permisos del archivo para ejecutar
chmod +x /usr/local/sbin/web-backup.sh

# iniciar servicios manualmente
systemctl start web-backup.timer web-backup.service

# check status de timer y servicio
systemctl status web-backup.timer
  Loaded: loaded (/etc/systemd/system/web-backup.timer; enabled; vendor preset: disabled)
   Active: active (waiting) since Mon 2023-08-14 10:33:07 EDT; 35s ago

systemctl status web-backup.service
● web-backup.service - Backup the web site, every day, or the boss will be mad
   Loaded: loaded (/etc/systemd/system/web-backup.service; enabled; vendor preset: disabled)
   Active: inactive (dead) since Mon 2023-08-14 10:33:07 EDT; 1min 17s ago
  Process: 4359 ExecStart=/usr/local/sbin/web-backup.sh (code=exited, status=0/SUCCESS)
 Main PID: 4359 (code=exited, status=0/SUCCESS)

# servidor listo para usarse


## Journal Control Utility: Administrar archivos y Troubleshooting


In [ ]:
# cambiar a cuenta root
sudo su -

# check el estado del servicio web
systemctl status httpd.service

# intentar iniciar el servicio web
systemctl start httpd.service

# check Journal
journalctl -u httpd.service

-- Logs begin at Mon 2023-08-14 10:10:57 EDT, end at Mon 2023-08-14 10:43:37 EDT. --
Aug 14 10:43:37 ip-10-0-1-102.ec2.internal systemd[1]: Starting The Apache HTTP Server...
Aug 14 10:43:37 ip-10-0-1-102.ec2.internal httpd[22861]: httpd: Could not open configuration fi
Aug 14 10:43:37 ip-10-0-1-102.ec2.internal systemd[1]: httpd.service: main process exited, code
Aug 14 10:43:37 ip-10-0-1-102.ec2.internal systemd[1]: Failed to start The Apache HTTP Server.
Aug 14 10:43:37 ip-10-0-1-102.ec2.internal systemd[1]: Unit httpd.service entered failed state.
Aug 14 10:43:37 ip-10-0-1-102.ec2.internal systemd[1]: httpd.service failed.

# revisar directorio donde la configuracion httpd debe estar
ls /etc/httpd/conf

httpd.conf.bkup  magic

# restaurar la configuracion httpd original del archivo  
mv /etc/httpd/conf/httpd.conf.bkup /etc/httpd/conf/httpd.conf

# reiniciar el servicio
systemctl restart httpd.service

# check el estado del servicio
systemctl status httpd.service

● httpd.service - The Apache HTTP Server
   Loaded: loaded (/usr/lib/systemd/system/httpd.service; disabled; vendor preset: disabled)
   Active: active (running) since Mon 2023-08-14 10:49:02 EDT; 39s ago
     Docs: man:httpd(8)
           man:apachectl(8)
 Main PID: 22879 (httpd)
   Status: "Total requests: 0; Current requests/sec: 0; Current traffic:   0 B/sec"
   CGroup: /system.slice/httpd.service
           ├─22879 /usr/sbin/httpd -DFOREGROUND
           ├─22880 /usr/sbin/httpd -DFOREGROUND
           ├─22881 /usr/sbin/httpd -DFOREGROUND
           ├─22882 /usr/sbin/httpd -DFOREGROUND
           ├─22883 /usr/sbin/httpd -DFOREGROUND
           └─22884 /usr/sbin/httpd -DFOREGROUND

# navegar a pagina web local 
elinks http://localhost

## Generar y cambiar SSH Keys para acceso seguro

### Crear ssh keys en servidor1 y servidor2

In [ ]:
# ingresar servidor1
ssh usuario@ IP_servidor1

# ver archivos de directorio
ls -la

# cambiar directorio a .ssh
cd .ssh

# ver archivos de directorio
ls -la

# generar ssh key para servidor1
ssh-keygen
# ver archivos de directorio
ls -la

# ver contenido de archivo  id_rsa.pub
cat id_rsa.pub


# ingresar servidor2
ssh usuario@ IP_servidor2

# cambiar directorio a .ssh
cd .ssh

# ver archivos de directorio
ls -la

# instalar editor de texto nano
sudo yum install nano
pass: password

# abrir archivo  authorized_keys
nano authorized_keys

# agregar key

control+x
yes


### Cambiar ssh key entre servidores

In [ ]:
# en servidor2
ssh-keygen

# ver archivos de directorio
ls -la

# ver contenido de archivo id_rsa.pub
cat id_rsa.pub

# copiar key
# salir de servidor2
exit

# instalar editor de texto nano en servidor
sudo yum install nano

# ver archivos de directorio
ls -la

# abrir archivo authorized_keys en archivo de texto nano 
nano authorized_keys



### Probar configuración

In [ ]:
# entrar a servidor2 desde servidor1
ssh usuario@ IP_servidor2

# entrar a servidor1 desde servidor2
ssh usuario@ IP_servidor1



## Crear y usar SSH tunel para tráfico de red

### ssh para cliente a servidor sin contraseña

In [ ]:
# ingresar a servidor CLIENT
ssh usuario@IP_servidor CLIENT

# generar key
ssh-keygen
ENTER

# copiar key de SSH Tunnel Server
ssh-copy-id cloud_user@IP_servidor Tunnel (privada)

# salir de servidor Tunnel
logout


### verificar accesibildad a ssh Tunnel 

In [ ]:
# crear SSH Tunnel
ssh -f cloud_user@10.0.1.100 -L 2000:10.0.1.100:80 -N

# usar comando curl para verificar accesibilidad
curl localhost:2000 

Webpage worked (check)

## Administrar seguridad con TCP Wrappers y Systemd Sockets



### Configurar sshd para usar Sockets

In [ ]:
# entrar a servidor
ssh cloud_user@IP_servidor

# verificar que unidad sshd.socket no esta activa
systemctl status sshd.socket


● sshd.socket - OpenSSH Server Socket
   Loaded: loaded (/usr/lib/systemd/system/sshd.socket; disabled; vendor preset: disabled)
   Active: inactive (dead)
     Docs: man:sshd(8)
           man:sshd_config(5)
   Listen: [::]:22 (Stream)
 Accepted: 0; Connected: 0


# set 'at job' que detiene la unidad sshd.service, iniciar sshd.socket
sudo at now + 3 minutes
pass: password

# agregar
at> systemctl stop sshd.service
at> systemctl start sshd.socket

# terminar configuracion 'at job'
control+D

# verificar que unidad sshd.socket esta activa y funcionando
systemctl status sshd.socket

# habilitar socket para SSH y deshabilitar servicio para SSH
sudo systemctl enable sshd.socket
sudo systemctl disable sshd.service

### Set TCP Wrapper para permitir solo SSH


In [ ]:
# verificar que el servidor sshd se compiló para usar TCP wrappers
ldd /usr/sbin/sshd | grep libwrap

# editar el archivo /etc/hosts.allow
sudo vim /etc/hosts.allow


# agregar linea de codigo al archivo
sshd2 sshd : ALL

# editar archivo /etc/hosts.deny
sudo vim /etc/hosts.deny

# agregar linea de codigo al archivo
ALL : ALL

# salir sesión ssh
exit

# reconectar a la sesión segura shell
ssh cloud_user@&IP_servidor



## Instalar y Administrar paquetes en Debian/Ubuntu

### Instalar paquetes en servidor web Apache

In [ ]:

# actualizar paquetes
sudo apt update

# instalar paquetes
sudo apt install apache2 wget



### Verificar que el servidor está funcionando y ver resultado


In [ ]:
# verificar servidor
curl http://localhost

# capturar salida de http request
wget --output-document=local_index.response http://localhost (local_index.response)

# quitar local_index.response
rm local_index.response

# mover index.html a  local_index.response
mv index.html local_index.response




## Instalar y Administrar paquetes en Red Hat/CentOS 

### Instalar aplicaciones E-links disponibles

In [ ]:

# entrar a servidor publico 
ssh usuario@IP_servidor

# ver archivos de directorio
ls -la

# ver información de carpeta 'Downloads'
Downloads/

# cambiar a directorio Downloads
cd Downloads/

# ver archivos de directorio
ls -la

# instalar el paquete e-links RPM
sudo rpm -i elinks-0.12-0.37.pre6.el7.0.1.x86_64.rpm
pass: password

# ver dependencia que falla y ver qué provee la dependencia
yum provides libmozjs185*

# instalar el paquete js
sudo yum install js
y

# intentar instalar el paquete e-links RPM
sudo rpm -i elinks-0.12-0.37.pre6.el7.0.1.x86_64.rpm

# ver dependencia que falla y ver qué provee la dependencia
yum provides */libnss_compat_ossl*

# instalar nss_compat_ossl
sudo yum install nss_compat_ossl

# intentar instalar el paquete e-links RPM
sudo rpm -i elinks-0.12-0.37.pre6.el7.0.1.x86_64.rpm

### Verificar paquetes E-links RPM existen

In [ ]:
# ejectutar E-links
elinks

# abrir website por URL
http://www.amazon.com

## Troubleshooting RPM

### Instalar telnet

In [ ]:
# entrar a servidor publico 
ssh usuario@IP_servidor

# convertirse en usuario root
sudo -i

# instalar paquete telnet
yum install -y telnet

# verificar integridad 
cd /var/lib/rpm/
/usr/lib/rpm/rpmdb_verify Packages

# mover carpeta 'Packages' a 'Packages.bad' y crear nueva base de datos RPM de 'Packages.bad'
mv Packages Packages.bad
/usr/lib/rpm/rpmdb_dump Packages.bad | /usr/lib/rpm/rpmdb_load Packages

# verificar integridad de nueva base de datos RPM
/usr/lib/rpm/rpmdb_verify Packages

# Query los pquetes instalados para errores
rpm -qa > /dev/null

# reconstruir la base de datos RPM
rpm -vv --rebuilddb

# instalar telnet
yum install -y telnet



### Actualizar Apache

In [ ]:
# intentar instalar Apache
yum install -y httpd

# editar el archivo /etc/yum.conf
vim /etc/yum.conf

# quitar la exclusión para httpd
exclude=httpd

# guardar y cerrar archivo
:wq 

# instalar Apache
yum install -y httpd

## Crear Estructura de Directorio en Linux

### Crear Directorio Padre

In [ ]:
# entrar a servidor  publico
ssh usuario@IP_servidor

# manual
mkdir -p Projects/ancient
mkdir Projects/classical
mkdir Projects/medieval

# expasion bash
mkdir -p Projects/{ancient,classical,medieval}

### Crear Sub-directorios

In [ ]:
# manual
mkdir Projects/ancient/egyptian
mkdir Projects/ancient/nubian
mkdir Projects/classical/greek
mkdir Projects/medieval/britain
mkdir Projects/medieval/japan


# expasion bash
mkdir Projects/ancient/{egyptian,nubian}
mkdir Projects/classical/greek
mkdir Projects/medieval/{britain,japan}


### Crear archivos vacios

In [ ]:
# crear archivos vacios
touch Projects/ancient/nubian/further_research.txt
touch Projects/classical/greek/further_research.txt



### Cambiar nombre de sub-directorio 

In [ ]:

mv Projects/classical Projects/greco-roman

## Archivos .Tar/Archivos Comprimidos

In [ ]:
# ingresar a servidor 

ssh usuario@IP_servidor

### Tamaño de archivo .txt original

In [ ]:
# ver tamaño original de archivo
ls -lh junk.txt

### Crear archivos .zip


In [ ]:
# comprimir archivo con gzip
gzip junk.txt

# ver tamaño de archivo
ls -lh

# descomprimir archivo .gzip
gunzip junk.txt.gz


# comprimir archivo con bzip2
bzip2 junk.txt

# ver tamaño de archivo
ls -lh junk.txt.bz2

# descomprimir archivo .gzip
bunzip2 junk.txt.bz2




# comprimir archivo con xz
xz junk.txt

# ver tamaño de archivo
ls -lh

# descomprimir archivo .gzip
unxz junk.txt.xz




### Crear archivos .tar

In [ ]:
# crear tarball con gzip
tar -cvzf gztar.tar.gz junk.txt

# crear tarball con bzip2
tar -cvjf bztar.tar.bz2 junk.txt

# crear tarball con xz
tar -cvJf xztar.tar.xz junk.txt

# comprar tamaños de archivos comprimidos
ls -lh


### Leer archivos comprimidos (texto)

In [ ]:
# copiar archivo /etc/passwd a directorio principal (home)
cp /etc/passwd /home/cloud_user/

# comprimir archivo .tar con gzip
tar -cvzf passwd.tar.gz passwd

# leer archivo comprimido 
zcat passwd.tar.gz

# comprimir arhivo a tarball con bzip2
tar -cvjf passwd.tar.bz2 passwd

# leer archivo comprimido 
bzcat passwd.tar.bz2

# comprimir arhivo a tarball con xz
tar -cvJf passwd.tar.xz passwd

# leer archivo comprimido
xzcat passwd.tar.xz


## Administrar atributos de archivos y Permisos

### Dar acceso al Directorio

In [ ]:
# entrar a servidor
ssh usuario@IP_servidor

# cambiar a directorio opt
cd /opt

# ver archivos y permisos de directorio
ls -la

# intentar acceder a directorio myapp
cd myapp/


# permisos de acceso solo para usuario  tkirk
# cambiar permisos de acceso
sudo chmod 777 myapp

# ver archivos y permisos de directorio
ls -la

# intentar acceder a directorio myapp con nuevos permisos
cd myapp/


### Cambiar **Permisos** del Directorio

In [ ]:
# quitar permisos de ejecucción(ejecutar)
sudo chmod -f -x -R *

# dar permisos de leer y escribir (read/write)
# también quitar permisos de ejecucción automáticamente
sudo chmod 666 -f -R *

# ver archivos y permisos de directorio
ls -la

# NOTA: para que los usuarios puedan navegar en los directorios, los directorios deben estar en ejecutable
sudo find /opt/myapp -type d -exec chmod o+x {} \;


# Bash

# Cronjob - Crontab

## Ansible Cron

In [ ]:
# ingresar a Cloud Server Node1
ssh username@IP_public_Node1

# convertir usuario en Ansible
su - ansible

# check que Ansible funciona via comando ad-hoc 
ansible all -m ping

# crear playbook 
# task sobre los nodos a las 5am y 5pm

archivo  cron-tasks.yml
---
- hosts: dbsystems
  user: ansible
  become: yes
  gather_facts: no
  tasks:
  - name: Ensure a job that runs at 5am and 5pm exists. Creates an entry like "0 5,17 * * * df -h >> /tmp/diskspace"
    cron:
      name: "Job 0001"
      minute: "0"
      hour: "5,17"
      job: "df -h >> /tmp/diskspace"


# ejecutar playbook 
ansible-playbook cron-tasks.yml

# corroborar que cron job existe en los nodos
sudo crontab -l

0 5,17 * * * df -h >> /tmp/diskspace



In [ ]:
# crear playbook delete-crontask.yml
# quitar el cron job  

delete-crontask.yml

---
- hosts: dbsystems
  user: ansible
  become: yes
  gather_facts: no
  tasks:
  - name: 'Ensure an old job is no longer present. Removes any job that is prefixed by "#Ansible: Job 0001" from the crontab'
    cron:
      name: "Job 0001"
      state: absent

In [ ]:
# modificar archivo cron-tasks.yml
---
- hosts: dbsystems
  user: ansible
  become: yes
  gather_facts: no
  tasks:
  - name: Ensure a job that runs at 5am and 5pm exists. Creates an entry like "0 5,17 * * * df -h >> /tmp/diskspace"
    cron:
      name: "Job 0002"
      minute: "0"
      hour: "4,17"
      job: "df -h >> /tmp/diskspace"


# ejecutar playbook cron-tasks.yml
ansible-playbook cron-tasks.yml

# log in a uno de los nodos 
# check 
sudo crontab -l

Job 0001
Job 0002

# desde maquina de control
# ejecutar playbook
ansible-playbook delete-crontask.yml


##  CRONJOB: Crear Cronjob para ejecutar script periódicamente



In [ ]:
# entrar a servidor
ssh username@IP_public

# convertir en usuario root
sudo -i

# verificar que servicio crond esta habilitado y ejecutandose
systemctl status crond.service


### Verificar que   /usr/local/bin/loadavg.sh esta ejecutable para todos y produce output correcta

In [ ]:
# ver permisos de archivo /usr/local/bin/loadavg.sh 
ls -l /usr/local/bin/loadavg.sh

# convertir archivo en ejecutable
chmod a+x /usr/local/bin/loadavg.sh

# ejecutar archivo 
/usr/local/bin/loadavg.sh

# verificar envio de datos a /var/log/loadavg.log
cat /var/log/loadavg.log


### Crear CRON JOB que ejecuta /usr/local/bin/loadavg.sh  una vez por minuto  Entre 8am-5pm de Lunes-Viernes

In [ ]:

# exportar editor de texto nano
export EDITOR=nano

# editar archivo crontab -e
crontab -e

# Min   Hour   DoM   Mon   DoW    Command

   *    8-17    *     *    1-5   /usr/local/bin/loadavg.sh
   *     *      *     *     *      echo hello

# ver contenido de archivo crontab -e
crontab -l

# verificar nuevamente que servicio crond esta habilitado y ejecutandose
systemctl status crond.service

# verificar que CRONJOB se ha ejecutado
tail /var/log/cron

# verificar CRONJOB está ejecutandose y produce output correcta
cat /var/log/loadavg.log


## CRONJOB: Sistema de Utilización de Logging con Cronograma con nmon

In [ ]:
# ingrear a servidor 
ssh username@IP_public

# instalar nmon
sudo apt install nmon

# ejecutar nmon
nmon


### Crear directorio log y confirmar accesibilidad

In [ ]:
# crear directorio nmon
mkdir nmon

# crear archivo en directorio nmon
touch /home/cloud_user/nmon/test/txt

# verificar archivo
ls -l nmon


### Verificar parámetros de Logging y confirmar contenido de Log file


In [ ]:
# ejectuar test de funcionalidad  nmon
nmon -f -s 2 -c 5 -m /home/cloud_user/nmon

# verificar existencia de archivo log file
ls -l nmon

# verificar contenidos de archivo log file
cat nmon/LOGFILENAME



### Crear CRONJOB para ejecutar el comando programado

In [ ]:

# instalar paquete
sudo apt install package

# abrir CRONTAB
crontab -e

# editar archivo crontab
0 22 * * 2 nmon -f -s 60 -c 480 -m /home/cloud_user/nmon/

# guardar archivo


## CRONJOB:  Instalar y Configurar AIDE ( Advanced Intrusion Detection Environmen)

### Instalar y Configurar AIDE

In [ ]:
# entrar a servidor 
ssh username@IP_public

# cambiar rol de usuario a root (super-user)
sudo su

# instalar AIDE
yum install -y aide

# inicializar AIDE 
/usr/sbin/aide --init

# copiar base de datos inicializada para produccion
cp /var/lib/aide/aide.db.new.gz /var/lib/aide/aide.db.gz



### Configurar AIDE con CRONJOB para ejecutar cada dia a 1am



In [ ]:
# crear CRONJOB
nano /etc/crontab

# ejecutar CRONJOB cada dia a 1am
# comando check: /usr/sbin/aide --check 
0 1 * * * /usr/sbin/aide --check 


# editar CRONJOB en nano , agregar después de encabezado 
# definir directorios para monitorear
nano /etc/aide.conf
/patient-data    DIR
/accounting     DIR

# agregar aplicacion para monitorear en cada acceso
# editar configuracion de AIDE
nano /etc/aide.conf
APP_ACCESS = a
/applications/payroll   APP_ACCESS


# actualizar base de datos de AIDE
/usr/sbin/aide --update

# copiar base de datos de AIDE actualizada
cp /var/lib/aide/aide.db.new.gz /var/lib/aide/aide.db.gz

## Configurar  CRONJOB en Kubernetes (containers)

In [ ]:
# entrar a servidor Kube Master 
ssh username@IP_public

# crear Bash script
vi cleanup-cronjob.yml

# 
apiVersion: batch/v1beta1
kind: CronJob
metadata:
  name: cleanup-cronjob
spec:
  schedule: "*/1 * * * *"
  jobTemplate:
    spec:
      template:
        spec:
          containers:
          - name: data-cleanup
            image: linuxacademycontent/data-cleanup:1
          restartPolicy: OnFailure


# crear CRONJOB en el cluster
kubectl apply -f ~/cleanup-cronjob.yml




## Configurar Timers con systemd

### Query y ver Información de Timer

In [ ]:

# ingresar a servidor
ssh username@IP_public

# ver timers activos
systemctl list-timers

# ver contenidos de archivo timer unit
systemctl cat fstrim.timer

# ver contenidos de archivo asociado unit 
systemctl cat fstrim.service

# analizar varias timer notations
systemd-analyze calendar hourly
systemd-analyze calendar daily
systemd-analyze calendar weekly
systemd-analyze calendar monthly
systemd-analyze calendar "*-*-* 09..17:00/5"
systemd-analyze calendar "Mon..Fri *-*-* 09..17:00/5"


# ejecutar comando
systemd-analyze calendar "*-*~03"


### Crear Timer, Servicio, y Script para ESCRIBIR Journal Entries

In [ ]:
# crear archivo timer unit
sudo systemctl edit --full --force blahwoof.timer

# editar archivo timer unit
[Unit]
Description=Run the blahwoof service every 5 seconds

[Timer]
OnCalendar=*-*-* *:*:0/5
AccuracySec=1s

[Install]
WantedBy=timers.target

# guardar y cerrar
control+X

# crear archivo asociado de servicio unit file
sudo systemctl edit --full --force blahwoof.service
# editar archivo asociado de servicio 
[Unit]
Description=Write a quick entry to the systemd journal

[Service]
Type=oneshot
User=cloud_user
ExecStart="/home/cloud_user/journo.sh"

[Install]
WantedBy=blahwoof.timer
# guardar y cerrar
control+X


# crear script
vim journo.sh
# editar script

#!/bin/bashs

echo "blahwoof entry at `date`" | systemd-cat

# lista de timers activos
systemctl list-timers

# permitir y comenzar timer
sudo systemctl enable blahwoof.timer

# comenzar timer
sudo systemctl start blahwoof.timer

# check estado de blahwoof.timer
systemctl status blahwoof.timer


# Troubleshooting
# ver dentro del System Journal 
journalctl --follow
control+C

# hacer script ejecutable
chmod a+x ~/journo.sh

# corroborar resultados
journalctl --follow




# Bash Scripts

## Variables en Bash

In [ ]:
# entrar a servidor
ssh username@IP_public

# examinar path actual
# extraer directorio actual
pwd

# ver archivos de directorio actual
ls

# ver archivos de directorio  script
ls scripts/

# ejecutar test script especificando el path
./scripts/test.sh

# ejecutar test script sin path
test.sh

# ver varaibles del ambiente
# ver variable PATH
env

# examinar variable PATH
echo $PATH

# ejecutar comando
# agregar path a HOME/scripts
PATH="$PATH:$HOME/scripts"


# ejecutar script test.sh SIN especificar path
test.sh

# ver contenidos de archivo .profile
cat.profile

# ver variable $PATH
echo $PATH

# hacer source el archivo .profile
source .profile

# modificar archivo ~/.profile para hacer cambio permanente
echo 'PATH="$PATH:$HOME/scripts"' >> ~/.profile

# hacer source el archivo .profile otra vez
source .profile

# ver variable $PATH
echo $PATH

# ejecutar el script test.sh sin especificar PATH
test.sh



## Escribir un Bash script

### Escribir script para hacer backup de los archivos solicitados 

In [ ]:
# ingresar a servidor
ssh username@IP_public

# ver directorio
ls

# ver archivos de carpeta work/
ls work/

# editar script backup (editor vim)
sudo vim backup.sh

# agregar nota echo 
echo "Creating backup directory" >> /home/cloud_user/backup_logs

# ejecutar comando mkdir
mkdir /home/cloud_user/work_backup

# ejecutar echos en actividad 
echo "Copying Files" >> /home/cloud_user/backup_logs

# copiar archivos en directorio de backup
cp -v /home/cloud_user/work/*  /home/cloud_user/work_backup/ >>  /home/cloud_user/backup_logs


echo "Finished Copying Files" >>  /home/cloud_user/backup_logs

# guardar archivo backup.sh y salir de editor vim
:wq


### Ejecutar script de backup y verificar que se hizo backup de los archivos

In [ ]:
# convertir script backup.sh en ejecutable
sudo chmod +x backup.sh

# ejecutar archivo backup.sh 
./backup.sh

# verificar que logs tiene back-up
ls

# ejecutar comando  cat para ver historial del directorio de backup
cat /home/cloud_user/backup_logs





## Regex en script de backup

### Modificar script de backup y agregar regex

In [ ]:
# entrar a servidor
ssh username@IP_public

# ver archivos y directorios
ls

# ver archivos en directorio work
ls work

# abrir archivo de backup (editor vim)
vim backup.sh

# crear regex (regular expression/patron) que encuentra los archivos en directorio  work  para hacer backup
# patron 'adent|financ'   adent  o financ
ls work | grep -E 'adent|financ'


# editar archivo backup (editor vim)
vim backup.sh

for i in $(ls work | grep -E 'adent|financ')
do
        cp -v /home/$USER/work/$i /home/$USER/work_backup >> /home/$USER/$MYLOG
done


# eliminar comando que hacia backup de todos los archivos
#cp -v /home/$USER/work/* /home/$USER/work_backup/ >> /home/$USER/$MYLOG

# guardar y salir (vim)
ESC
:wq




### Ejecutar script y verificaar los archivos que hicieron backup

In [ ]:
# ejecutar archivo backup
./backup.sh mylog

# ver archivos que hicieron backup
ls

# ver archivos que hicieron backup en directorio work_backup
ls work_backup

# ver que los archivos solicitados se copiaron
# ver timestamps (hora) de las acciones

cat mylog


## Incorporar Sustituciones en un script de Backup 

### Modificar script de back-up 

In [ ]:
# entrar a servidor servidor
ssh username@IP_public


# ver archivos en el directorio
ls- la

# ver archivos en directorio  work
ls work/

# abrir script de backup existente en editor vim
vim backup.sh

# usar comando date en sustitucion de comando para agregar un timestamp al logfile antes crear el directorio backup
echo "Timestamp before work is done $(date +"%D %T")" >> /home/$USER/$MYLOG

# usar comando  date  en sustitucion de comando para agregar un timestamp al logfile después de que los archivos hayan sido copiados
echo "Timestamp after work is done $(date +"%D %T")" >> /home/$USER/$MYLOG
# salir 
ESC
:wq



### Ejectura script y verificar  que archivos tienen backup con la información requerida

In [ ]:
# ejectuar script de backup para el logfile
./backup.sh mylog

# ver contenidos de log file
cat mylog



## Provisionando una imagen de Packer con Bash 

### Crear Provisionador Inline

In [ ]:

# ingresar a servidor
ssh  username@IP_servidor

# cambiar a directorio para Packer build
cd packer/bookapp/

# abrir  archivo packer.json
vim packer.json

# actualizar archivo con shell provisioner block

    ],
    "provisioners": [
        {
            "type": "shell",
            "inline": ["sudo apt-get update -y && sudo apt-get upgrade -y"]
        }
    ]



### Crear el script  Provisioner

In [ ]:

# actualizar archivo shell provisioner block
# agregar comandos que crean directorio /opt/provisioning

    ],
    "provisioners": [
        {
            "type": "shell",
            "inline": [
                "sudo apt-get update -y && sudo apt-get upgrade -y",
                "sudo mkdir /opt/provisioning",
                "sudo chown ubuntu:ubuntu /opt/provisioning"
            ]
        },
        {
            "type": "shell",
            "scripts": [
                "scripts/init.sh",
                "scripts/webdeploy.sh"
            ],
            "environment_vars": ["HOSTNAME=bookapp"],
            "remote_folder": "/opt/provisioning",
            "skip_clean": true
        }
    ]
}

### Test el archivo Packer

In [ ]:

# validar el archivo
packer validate packer.json

# test el archivo 
packer build packer.json

## Implementar Control de Bash Flow en un script de backup

### Modificar script de backup para agregar Trap solicitada

In [ ]:
# entrar a servidor 
ssh username@IP_public

# editar archivo de backup en editor vim
vim backup.sh

# agregar funcion   ctrlc
function ctrlc {
        rm -rf /home/$USER/work_backup
        rm -f /home/$USER/$MYLOG
        echo "Received Ctrl+C"
        exit 255
}

# habilitar trapon ctrlc
trap ctrlc SIGINT

### Modificar script de backup para contener el logic solicitado por parámetros 

In [ ]:
# agregar condicional if
if [ -z "$1" ]; then
        echo "You have failed to pass a parameter. Please try again."
        exit 255;
fi


### Ejecutar script y verificar que los archivos tienen backup y traping y que la lógica de parámetros funciona

In [ ]:
# agregar comando sleep (probar logica)
sleep 15
# guardar archivo

# ejecutar archivo backup
./backup.sh


# ejecutar archivo backup con mylog
./backup.sh mylog
# cancelar script
control+C

# verificar que se recibe  ^CReceived Ctrl+C

# verificar que no hay archivos leftovers
ls

# ejecutar archivo backup con mylog otra vez
./backup.sh mylog

# ver lista de archivos que hicieron backup
ls work_backup/

# verificar que los archivos de backup se copiaron y ver timestamps(hora) de acciones
cat mylog

# EC2

## Crear instancia de EC2 en VPC

### Crear VPC

In [ ]:
# acceder a servicio VPC->Your VPCs
# crear VPC: VPC Only, name tag= my-vpc, IPv4 CIDR block: 10.0.0.0/16
# valores predeterminados de  IPv6 CIDR block y Tenancy fields 
# crear VPC

### Crear Subnet publica

In [ ]:
# clic en menu Subnets
# crear Subnet: VPC ID: my-vpc, Subnet name: my-public-subnet, Availability Zone: us-east-1a, IPv4 CIDR block: 10.0.0.0/24
# crear subnet

### Crear Route Table y configurar Internet Gateway 

In [ ]:
# seleccionar Subnet my-public-subnet
# clic en Acciones -> Edit subnet settings
# checkbox para Enable auto-assign public IPv4 address
# Guardar

# seleccionar Internet Gateways
# crear Internet Gateway: name tag=my-internet-gateway
# Create internet gateway
# clic Acciones -> Attach to VPC
# seleccionar my-vpc
# clic en  Attach internet gateway

# seleccionar Route Tables
#  Route Tables
# create route table: name=publicRT, VPC:my-vpc
# create route table
# clic en Edit routes
# agregar Route: Destination=0.0.0.0/0, target= Internet Gateway, my-internet-gateway
# guardar cambios

# en menu Subnet associations 
# editar Edit subnet associations
# checkbox my-public-subnet
# guardar asociaciones


### Lanzar instancia de EC2 en Subnet

In [ ]:
# acceder a EC2 -> Instancias
# Lanzar Instancia
# en AMI, seleccionar Amazon Linux 2 AMI
# seleccionar  t2.micro
# Review and Launch > Launch
# crear key-pair Create a new key pair: nombre="my-keypair"
# descargar key-pair
# Launch Instances
# ver instancias

### Acceder a instancia de EC2

In [ ]:
# seleccionar Instancia cuando esté ejecutandose
# clic en Conectar
# en EC2 Instance Connect section, click Conectar

## EC2 Instance Bootstrapping

### Instalar software en webserver-01

In [ ]:
# Acceder a EC2
# menu Resources, clic en Instances(running)
# seleccionar instancia webserver-01, clic en Conectar
# seleccionar EC2 Instance Connect
# Conectar 
# desde terminal(consola) de webserver-01, ingresar con ssh a servidor con credenciales Cloud Server of webserver-01
# ssh usuario@IPpublic

# actualizar y mejorar paquetes
# sudo apt-get update && sudo apt-get upgrade -y
# instalar apache
# sudo apt-get install apache2 -y


# Acceder a EC2
# EC2 dashboard > Instances
# clic en   Instances (running)
# seleccionar webserver-01 
# seccion de Details, copiar Public IPv4 address y pegar en pagina web para acceder a Apache
# en terminal EC2 de webserver-01, instalar unzip
# sudo apt-get install unzip -y
# descargar AWS CLI
# curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
# descomprimir archivos
# unzip awscliv2.zip
# instalar AWS CLI
# sudo ./aws/install
# verificar version de AWS CLI 
# aws --version
# hacer archivo  index.html ejecutable con acceso a usuario
# sudo chmod 777 /var/www/html/index.html
# instancia de Availability Zone de servidor 
# curl http://169.254.169.254/latest/meta-data/placement/availability-zone
# agregar Availability Zone, instance ID, public IP, and local IP instance  al archivo index.html
echo '<html><h1>Bootstrap Demo</h1><h3>Availability Zone: ' > /var/www/html/index.html
curl http://169.254.169.254/latest/meta-data/placement/availability-zone >> /var/www/html/index.html
echo '</h3> <h3>Instance Id: ' >> /var/www/html/index.html
curl http://169.254.169.254/latest/meta-data/instance-id >> /var/www/html/index.html
echo '</h3> <h3>Public IP: ' >> /var/www/html/index.html
curl http://169.254.169.254/latest/meta-data/public-ipv4 >> /var/www/html/index.html
echo '</h3> <h3>Local IP: ' >> /var/www/html/index.html
curl http://169.254.169.254/latest/meta-data/local-ipv4 >> /var/www/html/index.html
echo '</h3></html> ' >> /var/www/html/index.html

# navegar a apache y refrescar pagina para ver cambios
# regresar a terminal e instalar mysql
# sudo apt-get install mysql-server -y


### Usar Bootstrap script para construir webserver-02

In [ ]:
# en  AWS Management Console acceder a EC2
# en EC2 dashboard, clic en Launch instances
# en sección  Launch an instance, en  Name and tags ingresar   webserver-02
# Scroll down a  Application and OS Images (Amazon Machine Image- AMI), seleccionar logo de Ubuntu, clic en  dropdown menu y seleccionar Ubuntu Server 20.04 LTS (HVM), SSD Volume Type
# Scroll down en  Instance type, seleccionar t3.micro
# en  Key pair (login), clic en  Proceed without a key pair (Not recommended) Default value
# en seccion r Network settings, clic en Edit
# Auto-assign public IP: seleccionar Enable
# Firewall (security groups):  Select existing security group
# Common security groups: EC2SecurityGroup
# en seccion  Advanced details, clic en dropdown
# Metadata version y seleccionar  V1 and V2 (token optional)
# en seccion User data pegar bootstrap script
#!/bin/bash
sudo apt-get update -y
sudo apt-get install apache2 unzip -y
sudo systemctl enable apache2
curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
unzip awscliv2.zip
sudo ./aws/install
echo '<html><h1>Bootstrap Demo</h1><h3>Availability Zone: ' > /var/www/html/index.html
curl http://169.254.169.254/latest/meta-data/placement/availability-zone >> /var/www/html/index.html
echo '</h3> <h3>Instance Id: ' >> /var/www/html/index.html
curl http://169.254.169.254/latest/meta-data/instance-id >> /var/www/html/index.html
echo '</h3> <h3>Public IP: ' >> /var/www/html/index.html
curl http://169.254.169.254/latest/meta-data/public-ipv4 >> /var/www/html/index.html
echo '</h3> <h3>Local IP: ' >> /var/www/html/index.html
curl http://169.254.169.254/latest/meta-data/local-ipv4 >> /var/www/html/index.html
echo '</h3></html> ' >> /var/www/html/index.html
sudo apt-get install mysql-server
sudo systemctl enable mysql
# clic en Launch instance

# clic en View all instances
# refrescar, esperar a que instancia webserver-02 haya pasado los status checks, clic en Connect
#  EC2 Instance Connect y click Connect
# check que el apache se instaló correctamente en servidor webserver-02
sudo systemctl status apache2
# verificar que apache 2 se esta ejecutandose
ps aux | grep apache

# verificar que los siguinetes comandos generan error, porque no se instalaron en el bootstrap script 
sudo systemctl status mysql
sudo systemctl status mysqld
ps aux | grep mysql
sudo systemctl start mysql
# usar curl para recuperar user-data
curl http://169.254.169.254/latest/user-data
# instalar mysql manualmente
sudo apt-get install mysql-server -y
# habilitar mysql
sudo systemctl enable mysql

### Usar Bootstrapscript para construir webserver-03

In [ ]:
# navegar a EC2 dashboard, clic en Launch instances
# en seccion  Launch an instance, en Name and tags :webserver-03
# Scroll down a Application and OS Images (Amazon Machine Image) AMI, seleccionar logo Ubuntu, clicc en dropdown menu y seleccionar  Ubuntu Server 20.04 LTS (HVM), SSD Volume Type
# scrolldown a Instance type, CLIC EN dropdown menu y seleccionar t3.micro
# en Key pair (login), clic en dropdown y select Proceed without a key pair (Not recommended) Default value
# en Network settings, clic en  Edit
Auto-assign public IP:  Enable
Firewall (security groups): Select existing security group
Common security groups:  EC2SecurityGroup
# en seccion Advanced details, clic en dropdown menu 
# seleccionar  Metadata version, seleccionar  V1 and V2 (token optional).
# en  User data pegar bootstrap script

#!/bin/bash
sudo apt-get update -y
sudo apt-get install apache2 unzip -y
sudo systemctl enable apache2
curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
unzip awscliv2.zip
sudo ./aws/install
echo '<html><h1>Bootstrap Demo</h1><h3>Availability Zone: ' > /var/www/html/index.html
curl http://169.254.169.254/latest/meta-data/placement/availability-zone >> /var/www/html/index.html
echo '</h3> <h3>Instance Id: ' >> /var/www/html/index.html
curl http://169.254.169.254/latest/meta-data/instance-id >> /var/www/html/index.html
echo '</h3> <h3>Public IP: ' >> /var/www/html/index.html
curl http://169.254.169.254/latest/meta-data/public-ipv4 >> /var/www/html/index.html
echo '</h3> <h3>Local IP: ' >> /var/www/html/index.html
curl http://169.254.169.254/latest/meta-data/local-ipv4 >> /var/www/html/index.html
echo '</h3></html> ' >> /var/www/html/index.html
sudo apt-get install mysql-server -y
sudo systemctl enable mysql
# clic en  Launch instance

# clic en View all instances
# clic en refrescar 
# seleccionar webserver-02 de instancias en EC2, copiar Public IPv4 address
# pegar IP address en pagina web, acceder al apache de webserver-3
# navegar a  EC2 dashboard, refrescar para ver estatus de webserver-03
# cuando webserver-03 haya pasado los check estatus, seleccionar la instancia y clic en Connect
# clic en  EC2 Instance Connect y click Connect para conectar a instancia EC2 de webserver-03 en nueva terminal 
# desde la terminal corroborar que apache se instalo correctamente
sudo systemctl status apache2
ps aux | grep apache
# verificar que mysql se instaló correctamente
systemctl status mysql
# confirmar el progreso de  mysql
ps aux | grep mysql
# verificar que AWS CLI se instaló
aws --version
3 navegar a EC2 dashboard y copiar Public IP para webserver-03
# pegar Public IP para webserver-03 en pagina web para corroborar que el apache funciona correctamente


## EC2 Roles y Instance Profiles con IAM 

### Crear Trust Policy y Role usando AWS CLI

In [ ]:
# Acceder a S3
# abrir bucket con nombre s3bucketlookupfiles
# seleccionar archivo labreferences.txt
# clic Actions > Download
# abrir archivo labreferences.txt

# Acceder a EC2>Instances
# copiar public IP de instancia  Bastion Host
# abrir terminal ssh, ingresar a bastion host via ssh
ssh usuario@PublicIP_Bastion
# aws configure
# Enter x 2
# region name: us-east-1
# format: json

# crear archivo trust_policy_ec2.json
nano trust_policy_ec2.json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {"Service": "ec2.amazonaws.com"},
      "Action": "sts:AssumeRole"
    }
  ]
}
# guardar y salir de editor nano 
# ^X, Y 

# crear role AIM  DEV_ROLE
aws iam create-role --role-name DEV_ROLE --assume-role-policy-document file://trust_policy_ec2.json
# crear Politica IAM con permisos read only a un bucket de S3
nano dev_s3_read_access.json
{
    "Version": "2012-10-17",
    "Statement": [
        {
          "Sid": "AllowUserToSeeBucketListInTheConsole",
          "Action": ["s3:ListAllMyBuckets", "s3:GetBucketLocation"],
          "Effect": "Allow",
          "Resource": ["arn:aws:s3:::*"]
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:Get*",
                "s3:List*"
            ],
            "Resource": [
                "arn:aws:s3:::<DEV_S3_BUCKET_NAME>/*",
                "arn:aws:s3:::<DEV_S3_BUCKET_NAME>"
            ]
        }
    ]
}
# ^X, Y 

# crear politica DevS3ReadAccess
# aws iam create-policy --policy-name DevS3ReadAccess --policy-document file://dev_s3_read_access.json
# copiar  policy ARN



### Crear Instance Profile y  Asignar Role a EC2 Instance

In [ ]:
# asignar politica a role con DevS3ReadAccess_POLICY_ARN
aws iam attach-role-policy --role-name DEV_ROLE --policy-arn "<DevS3ReadAccess_POLICY_ARN>"
# verificar asignacion de politica
aws iam list-attached-role-policies --role-name DEV_ROLE

# crear Instance Profile DEV_PROFILE
# aws iam create-instance-profile --instance-profile-name DEV_PROFILE
# agregar role a DEV_PROFILE: DEV_ROLE
# aws iam add-role-to-instance-profile --instance-profile-name DEV_PROFILE --role-name DEV_ROLE
# verficar configuracion
aws iam get-instance-profile --instance-profile-name DEV_PROFILE

# Asignar role  DEV_PROFILE a Instancia
# Acceder a  EC2 > Instances
# copiar  instance ID de instancia Web Server
# en terminal ssh asignar DEV_PROFILE a instancia EC2
aws ec2 associate-iam-instance-profile --instance-id <LAB_WEB_SERVER_INSTANCE_ID> --iam-instance-profile Name="DEV_PROFILE"
# verficar configuracion con Web Server instance ID
aws ec2 describe-instances --instance-ids <LAB_WEB_SERVER_INSTANCE_ID>

### Test Permisos de S3 via AWS CLI (ssh)

In [ ]:
# abrir nueva terminal ssh
# acceder a instancia de web server instance via SSH
ssh cloud_user@<WEB_SERVER_PUBLIC_IP>
# verificar que la instancia asume el role DEV_ROLE
aws sts get-caller-identity
# ver lista de buckets en cuenta
# aws s3 ls
# copiar nombre de bucket: s3bucketdev
# ver archivos en s3bucketdev-
aws s3 ls s3://<s3bucketdev-123>

### Crear Politica IAM y Role

In [ ]:
# Acceder a  IAM > Policies
# crear politica  Create policy
# pestaña JSON
{
    "Version": "2012-10-17",
    "Statement": [
        {
          "Sid": "AllowUserToSeeBucketListInTheConsole",
          "Action": ["s3:ListAllMyBuckets", "s3:GetBucketLocation"],
          "Effect": "Allow",
          "Resource": ["arn:aws:s3:::*"]
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:Get*",
                "s3:List*"
            ],
            "Resource": [
                "arn:aws:s3:::<PROD_S3_BUCKET_NAME>/*",
                "arn:aws:s3:::<PROD_S3_BUCKET_NAME>"
            ]
        }
    ]
}
# Next
# nombre de politica  ProdS3ReadAccess
# clic en Create policy


# clic en Roles
# crear role Create role
# en scrolldown  Choose a use case: EC2.
# Next
# buscar politica  ProdS3ReadAccess
# clic checkbox para seleccionar ProdS3ReadAccess
# Next
# asignar nombre a role PROD_ROLE
# clic en crear role Create role

### Asignar Role IAM  una instancia EC2 

In [ ]:
# Acceder a  EC2 > Instances
# seleccionar instancia de  Web Server
# clic en Actions > Security > Modify IAM role
# en menu dropdown  IAM role dropdown, seleccionar PROD_ROLE
# actualizar Update IAM role

# abrir terminal, ingresar por ssh a  Web Server instance
# determinar identidad de rol actual aws sts get-caller-identity
# ver lista de buckets 
aws s3 ls
# copiar nombre de bucket s3bucketprod
# ver archivos de bucket 
aws s3 ls s3://<s3bucketprod-123>
# copiar nombre de bucket s3bucketsecret
# ver archivos de bucket secreto (acceso denegado)
aws s3 ls s3://<s3bucketsecret-123>

## Reducir costos de almacenamiento con EFS

### Crear archivo EFS

In [ ]:
# Acceder a EC2
# en sección de recursos Resources, seleccionar Instances (running)
# clic checkbox webserver-01
# seleccionar pestaña Storage y ver el disco adjunto de 10GB al volumen

# Acceder a EFS
# crear un archivo de sistema Create file system
# Name:SharedWeb, Virtual Private Cloud (VPC): VPC
# clic en Customize
# Storage class:  One Zone
# Availability Zone:  us-east-1a 
# Next > Next > Next > Create 
# clic en View file system 
# seleccionar pestaña Network
# cuando estatus es available, clic en Manage
# en sección Security groups, quitar grupo de seguridad adjunto 
# usar dropdown menu para seleccionar   EC2SecurityGroup (no default)
# clic en Guardar

# Acceder a EC2
# seleccionar Security Groups
# clic checkbox de grupo de seguridad no-default para ver los detalles del grupo de seguridad
# seleccionar pestaña Inbound rules
# clic en Edit Inbound rules
# clic en  Add rule
# Type: NFS
# Source: 0.0.0.0/0
# clic en  Save rules
# clic en  EC2 Dashboard
# seleccionar Instances(running)
# seleccionar webserver-01, conectar a webserver-01
# clic en Connect

### Montar archivo EFS y hacer test 

In [ ]:
# desde terminal EC2 de webserver-01
# ver lista de block devices
# lsblk
# ver data en disco de 10GB 
ls /data
# crear directorio para adjuntar volumen EFS
sudo mkdir /efs
# navegar a pestaña EFS que muestra el archivo SharedWeb
# clic en adjuntar Attach
# seleccionar Mount via IP
# copiar el comando NFS a clipboard
# navegar a terminal EC2 y pegar comando, editar reemplazando efs por /efs
sudo mount -t nfs4 -o nfsvers=4.1,rsize=1048576,wsize=1048576,hard,timeo=600,retrans=2,noresvport 10.0.0.47:/ /efs
# Enter


# ver volumen EFS montado
ls /efs
# ver lista de  block devices
lsblk
# ver archivos montados
mount
# ver archivos de sistema montados
df -h
# mover todos los archivos de /data a /efs 
sudo rsync -rav /data/* /efs
# ver archivos en /efs
ls /efs

### Quitar data vieja

In [ ]:
# quitar data de webserver-01
# desmontar particion
sudo umount /data
# abrir archivo /etc/fstab en editor de texto
sudo nano /etc/fstab
# quitar linea que empieza con UUID con Ctrl+K 
# construir un nuevo mount point
# navegar a pestaña EFS y asegurar que el dialogo de Attach esta abierto
# copiar direccion IP
# navegar a terminal y pegar direccion IP
<NFS MOUNT IP>:/ 		/data 	nfs4 	 <OPTIONS> 0 0
# guardar y salir Ctrl+X, Y
# desmontar /efs 
sudo umount /efs 
# ver archivos de sistema
df -h
# montar todo lo que no esta montado aun
sudo mount -a
# ver archivos del sistema, corroborar la direccion IP montada
df -h
# ver contenido de data
ls /data

# quitar volumen EBS adjunto a webserver-01
# navegar a EC2
# clic en Volumenes
# clic checkbox junto a volumen de 10GB adjunto a webserver-01
# seleccionar Actions y Detach volume
# Click Detach
# clic checkbox junto a volumen desacoplado
# usar Actions>Delete volume para borrar volumen

# quitar data de EC2 webserver-02 y webserver-03 (se repite el procedimiento para ambas instancias)
# Acceder a EC2>Instances
# clic checkbox junto a webserver-02
# clic en Connect 
# navegar a terminal webserver-01
# ver contenidos de  /etc/fstab
cat /etc/fstab
# copiar  mount point 
<NFS MOUNT IP>:/ 		/data 	nfs4 	 <OPTIONS> 0 0
# navegar a terminal de webserver-2 ( o webserver-3)
# desmontar la particion de data 
sudo umount /data
# abrir /etc/fstab en editor de texto
sudo nano /etc/fstab
# quitar linea con UUID
# pegar linea
<NFS MOUNT IP>:/ 		/data 	nfs4 	 <OPTIONS> 0 0
# salir y guardar
# montar particion
sudo mount -a
# ver archivos de sistema
df -h
# ver contenidos de /data
ls /dataa


# quitar volumen EBS adjunto a EC2
# navegar a EC2
# seleccionar Volumenes
# checkbox de los volumenes adjuntos a webserver-2 y  webserver-3
# usar Actions>Detach volume
# checkbox de los volumenes desacoplados a webserver-2 y  webserver-3
# usar Actions>Delete volume

# S3

# VPC

# IAM

## Crear y asumir roles

### Crear politica IAM

In [ ]:
# Acceder a servicio S3, ver buckets
# Acceder a servicio IAM
# En 

# CloudFormation

# Kubernetes